In [1]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path
    from io import BytesIO
    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/internationaltrade/datasets/uktradeinservicesallcountriesnonseasonallyadjusted/current/tradeinservicesbycountry.xlsx'
    inputFile = sourceFolder / 'tradeinservicesbycountry.xlsx'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)    

In [2]:
next_table = pd.DataFrame()

In [3]:
%%capture

%run "Trade in Services by Country .ipynb"
next_table = pd.concat([next_table, new_table])

%run "UK trade in services by partner country.ipynb"
next_table = pd.concat([next_table, new_table])

In [4]:
from pathlib import Path
out = Path('out')
out.mkdir(exist_ok=True)
next_table.to_csv(out / 'observations.csv', index = False)

In [5]:
# scraper.dataset.family = 'Trade'
# with open(out / 'dataset.trig', 'wb') as metadata:
#     metadata.write(scraper.generate_trig())